In [59]:
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CDSView, IndexFilter, FactorRange

output_notebook()

usa_census = pd.read_csv(
    './data/USA/DECENNIALDDHCA2020.T01001_2024-12-15T154541/DECENNIALDDHCA2020.T01001-Data.csv'
)

usa_census.drop(index=usa_census.index[0], axis=0, inplace=True)

usa_census.drop(usa_census.columns[5], axis=1, inplace=True)
usa_census.columns = usa_census.columns.str.lower()
usa_census.rename(columns={'t01001_001n':'total'}, inplace=True)

singular_popgroups = usa_census['popgroup_label'][usa_census['popgroup_label'].str.match('.* alone$')]
singular_popgroups = singular_popgroups.str.replace(' alone', '')
usa_census['in_any_combination'] = usa_census['popgroup_label'].str.match('.*in any combination$')

usa_census["popgroup"] = usa_census['popgroup_label'] \
    .apply(lambda popgroup_label: [search_word for search_word in singular_popgroups if search_word in popgroup_label]) \
    .str[0]

usa_census.groupby("popgroup")
usa_census_total_x = usa_census[usa_census.total == 'X']
usa_census = usa_census[usa_census.total != 'X']
usa_census['total'] = usa_census['total'].astype(int)
usa_census.sort_values('total', ascending=False, inplace=True)

p = figure(
    title="Histogram of Population Totals",
    x_axis_label='Popgroup Label',
    y_axis_label='Total',
    x_range=usa_census['popgroup_label'][0:10]
    )

p.xaxis.major_label_orientation = 3.14159/4

p.vbar(x=usa_census['popgroup_label'][0:10], top=usa_census['total'][0:10], width=0.5)

show(p)


Loading BokehJS ...